In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import norm,skew
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [3]:
train=pd.read_csv('F:\\study\\ml\\DataSet\\House_Prices\\train.csv')
test=pd.read_csv('F:\\study\\ml\\DataSet\\House_Prices\\test.csv')

In [4]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


delete outlier data and drop id

In [5]:
train=train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)

In [6]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [7]:
train_id=train['Id']
test_id=test['Id']

In [8]:
train.drop('Id',axis=1,inplace=True)
test.drop('Id',axis=1,inplace=True)

- fit norm dist plot

- - plot Q-Q 
- double check skew >0

 - log transform target variable

In [16]:
train['SalePrice']=np.log1p(train['SalePrice'])
(mu,sigma)=norm.fit(train['SalePrice'])
stats.probplot(train['SalePrice'],plot=plt)
plt.show()

- features engineing

In [17]:
ntrain=train.shape[0]
ntest=test.shape[0]
y_train=train.SalePrice.values
all_data=pd.concat((train,test)).reset_index(drop=True)
all_data.drop(['SalePrice'],axis=1,inplace=True)
print('all data shape:',all_data.shape)

all data shape: (2917, 79)


- Missing Data

In [18]:
all_data_na=(all_data.isnull().sum()/len(all_data)) * 100
all_data_na=all_data_na.drop(all_data_na[all_data_na==0].index).sort_values(ascending=False)[:30]
missing_data=pd.DataFrame({'Missing Ratio': all_data_na})
missing_data.head(20)

,Missing Ratio
PoolQC,99.691464
MiscFeature,96.400411
Alley,93.212204
Fence,80.425094
FireplaceQu,48.680151
LotFrontage,16.660953
GarageQual,5.450806
GarageCond,5.450806
GarageFinish,5.450806
GarageYrBlt,5.450806


In [19]:
plt.subplots(figsize=(12,10))
plt.xticks(rotation='90')
sns.barplot(x=all_data_na.index,y=all_data_na)
plt.xlabel('Features',fontsize=15)
plt.ylabel('Precent of missing value',fontsize=15)
plt.title('Precent of miss data by feature',fontsize=15)
plt.show()

- Data Correlation

- imputing missing values

In [23]:
for i in all_data.groupby('Neighborhood')['LotFrontage']:
    print(i)

('Blmngtn', 219     43.0
229     43.0
385     43.0
443     53.0
465      NaN
558      NaN
596     53.0
638     53.0
789     43.0
850      NaN
981     43.0
1003    43.0
1018    43.0
1022    43.0
1125    53.0
1392    53.0
1413    51.0
1481    43.0
1683    43.0
1982     NaN
1983     NaN
1984    53.0
2318     NaN
2319    53.0
2669    43.0
2670     NaN
2671     NaN
2672    43.0
Name: LotFrontage, dtype: float64)
('Blueste', 598     24.0
955     24.0
1596    35.0
1597    35.0
1905    24.0
1906    24.0
1907    24.0
2207    24.0
2567    24.0
2569    35.0
Name: LotFrontage, dtype: float64)
('BrDale', 225     21.0
227     21.0
232     21.0
235     21.0
363     21.0
430     21.0
432     24.0
500     21.0
654     21.0
836     21.0
1028    21.0
1103    24.0
1218    21.0
1290    21.0
1332    24.0
1376    21.0
1469    21.0
1470    21.0
1649    21.0
1650    21.0
1959    21.0
1960    21.0
1961    21.0
2280    21.0
2281    21.0
2640    24.0
2641    24.0
2642    21.0
2643    21.0
2644    21.0
Name: LotFr

In [24]:
all_data['PoolQC']=all_data['PoolQC'].fillna('None')
all_data['MiscFeature']=all_data['MiscFeature'].fillna('None')
all_data['Alley']=all_data['Alley'].fillna('None')
all_data['Fence']=all_data['Fence'].fillna('None')
all_data['FireplaceQu']=all_data['FireplaceQu'].fillna('None')
all_data['LotFrontage']=all_data.groupby('Neighborhood')['LotFrontage'].transform(lambda x:x.fillna(x.median()))

In [25]:
for col in ['GarageType','GarageFinish','GarageQual','GarageCond']:
    all_data[col]=all_data[col].fillna('None')

In [26]:
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)

In [27]:
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)

In [28]:
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')

In [29]:
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)

In [30]:
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])

In [31]:
all_data = all_data.drop(['Utilities'], axis=1)

In [32]:
all_data["Functional"] = all_data["Functional"].fillna("Typ")

In [33]:
all_data['Electrical']=all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])

In [34]:
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])

In [35]:
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])

In [36]:
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")

In [37]:
all_data_na=(all_data.isnull().sum())/len(all_data) * 100
all_data_na=all_data_na.drop(all_data_na[all_data_na==0].index).sort_values(ascending=False)
miss_data=pd.DataFrame({'Missing Ratio':all_data_na})
miss_data.head()

,Missing Ratio


In [38]:
all_data['MSSubClass'].dtype

dtype('int64')

In [39]:
all_data['MSSubClass']=all_data['MSSubClass'].apply(str)

In [40]:
all_data['MSSubClass'].dtype

dtype('O')

In [41]:
all_data['OverallCond'] = all_data['OverallCond'].astype(str)

In [42]:
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

In [43]:
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond',
        'ExterQual', 'ExterCond', 'HeatingQC', 'PoolQC', 'KitchenQual',
        'BsmtFinType1', 'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure',
        'GarageFinish', 'LandSlope', 'LotShape', 'PavedDrive', 'Street',
        'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 'YrSold', 'MoSold')
for c in cols:
    lbl=LabelEncoder()
    lbl.fit(list(all_data[c].values))
    all_data[c]=lbl.transform(list(all_data[c].values))

In [44]:
all_data.shape

(2917, 78)

In [45]:
all_data['TotalSF']=all_data['TotalBsmtSF']+all_data['1stFlrSF']+all_data['2ndFlrSF']

- skewed features

In [46]:
numeric_feats=all_data.dtypes[all_data.dtypes != 'object'].index
skewed_feats=all_data[numeric_feats].apply(lambda x : skew(x.dropna())).sort_values(ascending=False)
skewness=pd.DataFrame({'Skew':skewed_feats})
skewness

,Skew
MiscVal,21.939672
PoolArea,17.688664
LotArea,13.109495
LowQualFinSF,12.084539
3SsnPorch,11.372080
LandSlope,4.973254
KitchenAbvGr,4.300550
BsmtFinSF2,4.144503
EnclosedPorch,4.002344
ScreenPorch,3.945101


- get abs(skew ) > 0.75

In [47]:
skewness

,Skew
MiscVal,21.939672
PoolArea,17.688664
LotArea,13.109495
LowQualFinSF,12.084539
3SsnPorch,11.372080
LandSlope,4.973254
KitchenAbvGr,4.300550
BsmtFinSF2,4.144503
EnclosedPorch,4.002344
ScreenPorch,3.945101


In [48]:
skewness=skewness[abs(skewness['Skew'])>0.75]

In [49]:
from scipy.special import boxcox1p
from scipy.stats import boxcox,boxcox_normmax

In [50]:
for i in range(len(skewness)):
    print(skewness.index[i],'-----skew:',skewness.values[i],'-----lm:',boxcox_normmax(all_data[skewness.index[i]]+1))
    

MiscVal -----skew: [21.93967217] -----lm: 0.01726260601513818
PoolArea -----skew: [17.68866449] -----lm: -0.04560524017392778
LotArea -----skew: [13.10949469] -----lm: 0.08821182373699077
LowQualFinSF -----skew: [12.0845388] -----lm: -0.07663224690203845
3SsnPorch -----skew: [11.37207993] -----lm: 0.0701437654661614
LandSlope -----skew: [4.97325361] -----lm: -0.6069157851779312
KitchenAbvGr -----skew: [4.30055011] -----lm: 0.2281310475189893
BsmtFinSF2 -----skew: [4.14450336] -----lm: 0.16704477665499903
EnclosedPorch -----skew: [4.00234409] -----lm: 0.3258432881427254
ScreenPorch -----skew: [3.94510123] -----lm: 0.4855404906678094
BsmtHalfBath -----skew: [3.92999597] -----lm: 0.07264586855126172
MasVnrArea -----skew: [2.6217193] -----lm: 0.418769251495698
OpenPorchSF -----skew: [2.5293582] -----lm: 0.45209346897143926
WoodDeckSF -----skew: [1.84479163] -----lm: 0.6307651224816825
1stFlrSF -----skew: [1.25728598] -----lm: -0.03890776331837051
LotFrontage -----skew: [1.1030386] -----lm:

- - change fixed lam=0.15 to different lam

In [51]:
skewed_feats=skewness.index
for feat in skewed_feats:
    lm=boxcox_normmax(all_data[feat]+1)
    all_data[feat]=boxcox1p(all_data[feat],lm)

In [52]:
all_data.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,...,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,TotalSF
0,5.939033,1025.651978,0.0,1,3,1Fam,7.746519,15.148815,144.117862,0.0,...,WD,0.0,0.46657,8,856.0,0.000000,2003,2003,2,35.576152
1,6.234989,0.000000,0.0,1,3,1Fam,7.746519,2.029476,181.719186,0.0,...,WD,0.0,0.46657,6,1262.0,56.184225,1976,1976,1,35.371886
2,5.994335,1040.521059,0.0,1,3,1Fam,7.746519,6.781765,110.441033,0.0,...,WD,0.0,0.46657,6,920.0,0.000000,2001,2002,2,36.241071
3,6.027703,904.477422,0.0,1,3,1Fam,1.279612,15.148815,61.795315,0.0,...,WD,0.0,0.46657,7,756.0,0.000000,1915,1970,0,35.120723
4,6.161220,1273.024862,0.0,1,4,1Fam,7.746519,0.000000,136.624601,0.0,...,WD,0.0,0.46657,9,1145.0,42.245703,2000,2000,2,39.002317


In [55]:
all_data=pd.get_dummies(all_data)
all_data.shape

(2917, 220)

In [56]:
train=all_data[:ntrain]
test=all_data[ntrain:]

#### Model

In [57]:
from sklearn.linear_model import ElasticNet,Lasso,BayesianRidge,LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator,TransformerMixin,RegressorMixin,clone
from sklearn.model_selection import KFold,cross_val_score,train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [58]:
n_folds=5

def rmsle_cv(model):
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(train.values)
    rmse=np.sqrt(-cross_val_score(model,train.values,y_train,scoring='neg_mean_squared_error',cv=kf))
    return rmse

In [59]:
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))

In [60]:
ENet=make_pipeline(RobustScaler(),ElasticNet(alpha=0.0005,l1_ratio=0.9,random_state=3))

In [61]:
KRR=KernelRidge(alpha=0.6,kernel='polynomial',degree=2,coef0=2.5)

In [62]:
GBoost = GradientBoostingRegressor(n_estimators=3000,
                                   learning_rate=0.05,
                                   max_depth=4,
                                   max_features='sqrt',
                                   min_samples_leaf=15,
                                   min_samples_split=10,
                                   loss='huber',
                                   random_state=5)

In [63]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [64]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [65]:
score=rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 0.1099 (0.0064)



In [66]:
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

ElasticNet score: 0.1098 (0.0065)



In [70]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [71]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

[17:43:17] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:43:20] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:43:23] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings bu

In [72]:
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

LGBM score: 0.1161 (0.0069)

